In [1]:
import os

In [2]:
%pwd

'/home/muhammed-shafeeh/AI_ML/End_to_end_ml_project_Mlflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd


'/home/muhammed-shafeeh/AI_ML/End_to_end_ml_project_Mlflow'

In [5]:
import dagshub
dagshub.init(repo_owner='shafeeh011', repo_name='my-first-repo', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as shafeeh011

Initialized MLflow to track repo "shafeeh011/my-first-repo"

Repository shafeeh011/my-first-repo initialized!

🏃 View run skittish-lynx-267 at: https://dagshub.com/shafeeh011/my-first-repo.mlflow/#/experiments/0/runs/880295e303384b67adbfc6653a44e441
🧪 View experiment at: https://dagshub.com/shafeeh011/my-first-repo.mlflow/#/experiments/0


In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_file_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories,save_json

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluatiion_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])

        model_eval_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_file_path=config.test_file_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_path,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/shafeeh011/my-first-repo.mlflow",
        )

        return model_eval_config

In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import joblib
import mlflow
import mlflow.sklearn
from urllib.parse import urlparse

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_file_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")


        
        

In [13]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluatiion_config()
    model_eval_config = ModelEvaluation(config=model_eval_config)
    model_eval_config.log_into_mlflow()
except Exception as e:
    raise e


[2025-02-06 19:48:43,319: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-02-06 19:48:43,322: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-06 19:48:43,325: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-06 19:48:43,326: INFO: common: created directory at: artifacts]
[2025-02-06 19:48:43,328: INFO: common: created directory at: artifacts/model_evaluation]
[2025-02-06 19:48:43,703: INFO: common: json file saved at: artifacts/model_evaluation/metric.json]


2025/02/06 19:48:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/02/06 19:48:52 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 9
Created version '9' of model 'ElasticnetModel'.


🏃 View run persistent-ram-941 at: https://dagshub.com/shafeeh011/my-first-repo.mlflow/#/experiments/0/runs/9031ed5d95e0467fbb063ee9ad4dd3fe
🧪 View experiment at: https://dagshub.com/shafeeh011/my-first-repo.mlflow/#/experiments/0
